In [5]:
%autosave 900 

Autosaving every 900 seconds


(The above changes autosave to every 15 minutes<a id = autosave_time>[1]</a>, since (following feedback from previous assignment) I have things set up to automatically commit every time a file within the project directory is changed<a id = my_force_commit_script>[2]</a>, and a 2 minute autosave would be ridiculous.)

#### References

[1](#autosave_time): https://www.webucator.com/blog/2016/03/change-default-autosave-interval-in-ipython-notebook/

[2](#my_force_commit_script): https://stackoverflow.com/a/59121121/12462056